# Backticks Code Replacement Sandbox

The purpose of this notebook is to explore the functionality described in [this issue](https://github.com/open-resources/problem_bank_scripts/issues/35), started in [this PR](https://github.com/open-resources/problem_bank_scripts/pull/36) and continued [in this one](https://github.com/open-resources/problem_bank_scripts/pull/42).

For example, this
```
data2["params"]["part1"]["ans1"]["value"] = "Hello `x` world"
```

should get replaced with:
```
data2["params"]["part1"]["ans1"]["value"] = "Hello <code>x</code> world"
```

Having something we can concretely test will also help us debug future cases where the current functionality breaks and easily adapt/change the behaviour of this function as the PrairieLearn markdown rendering functionality improves.


## Function that does the conversion

In [3]:
def backticks_to_code_tags(html):
    """
    Describe functionality...

    """

    return html

## Case 1: single case of backticks:

Test cases:

- "Hello `x+1` world"
- "`x+1`"
- `x+1`

In [13]:
# Test backticks_to_code_tags here

## Case 2: multiple instances of backticks

Test cases:


- "Hello `x+1` and `y+5` world"
- `x+1` `y+5`


In [14]:
# Test backticks_to_code_tags here

## Case 3: Codefence

Test cases:

- see multi-line string below

In [15]:
"""
```
for i in range(5):

    print(i)
    print(f'hello work {i}')

```"""

"\n```\nfor i in range(5):\n\n    print(i)\n    print(f'hello work {i}')\n\n```"

In [16]:
""" Code block option:
```
for i in range(5):

    print(i)
    print(f'hello work {i}')

```
"""

" Code block option:\n```\nfor i in range(5):\n\n    print(i)\n    print(f'hello work {i}')\n\n```\n"

In [17]:
""" Code block option:
```
for i in range(5):

    print(i)
    print(f'hello work {i}')

```
Code block option end
"""

" Code block option:\n```\nfor i in range(5):\n\n    print(i)\n    print(f'hello work {i}')\n\n```\nCode block option end\n"